In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import srsly
from datasets import load_from_disk
from matplotlib import ticker
from scipy.special import entr, softmax
from sklearn.manifold import TSNE
from tqdm.auto import tqdm

In [ ]:
path_to_experiment = Path("../outputs/multirun/reinit_effect/patch")
path_to_experiment

In [ ]:
data = []

for dataset in path_to_experiment.iterdir():
    for strategy in dataset.iterdir():
        for reinit in strategy.iterdir():
            for run in reinit.iterdir():
                data_seed, model_seed = run.name.split("_")
                results = pd.DataFrame(
                    srsly.read_jsonl(run / "logs" / "test" / "epoch_level.jsonl")
                ).assign(
                    data_seed=int(data_seed.split("=")[1]),
                    model_seed=int(model_seed.split("=")[1]),
                    dataset=dataset.name,
                    strategy=strategy.name,
                    reinit=eval(str(reinit.name).split("=")[-1]),
                )
                data.append(results)

In [ ]:
df = pd.concat(data)
df

In [ ]:
df.groupby(["strategy", "reinit", "dataset"])[["data_seed", "model_seed"]].size()

In [ ]:
df_agg = (
    df.groupby(["dataset", "strategy", "reinit", "round"])[
        ["accuracy", "f1_macro", "precision_macro", "recall_macro"]
    ]
    .agg(["mean", "std", "min", "max"])
    .stack(0)
    .reset_index()
    .rename(columns={"level_4": "metric"})
)

In [ ]:
plot_data = df_agg.loc[
    (df_agg["dataset"] == "imdb") & (df_agg["metric"] == "f1_macro")
].rename(columns={"reinit": "re-initialise"})
plot_data

In [ ]:
sns.diverging_palette(250, 30, l=65, center="dark", n=3)

In [ ]:
def plot(plot_data, dpi, ymin, hline, title, metric):
    # colors = sns.color_palette("Spectral", n_colors=len(plot_data["strategy"].unique()))
    colors = sns.diverging_palette(
        250, 30, l=65, center="dark", n=plot_data["strategy"].nunique()
    )

    # sns.set_style("white", rc={"xtick.bottom" : True, "ytick.left" : True})
    plt.style.use("bmh")
    sns.set_context("paper")

    # plot
    fig, ax = plt.subplots()
    fig.dpi = dpi

    sns.lineplot(
        data=plot_data,
        x="round",
        y="mean",
        hue="strategy",
        style="re-initialise",
        palette=colors,
        ax=ax,
    )

    # fill area between min and max for each group
    for group1, group1_data in plot_data.groupby("strategy"):
        group_color = colors[plot_data["strategy"].unique().tolist().index(group1)]

        for _, group2_data in group1_data.groupby("re-initialise"):
            group2_data

            ax.fill_between(
                group2_data["round"],
                group2_data["min"],
                group2_data["max"],
                alpha=0.2,
                color=group_color,
            )

    # add 90% line
    ax.hlines(hline, *plt.xlim(), colors="red")

    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(5))
    ax.set_xlim(0, 100)
    ax.set_ylim(ymin)

    fig.suptitle(title)
    ax.set_title("(25 instances per round, 10 assigned to validation)", fontsize=10)
    ax.set_ylabel(metric)
    ax.set_xlabel("Labelling round")
    ax.legend(fontsize=10, bbox_to_anchor=(1, 1))

    sns.despine()
    plt.show()

In [ ]:
for dataset in ("agnews", "imdb"):
    if dataset == "imdb":
        continue
    for metric in ("f1_macro", "accuracy"):
        plot_data = df_agg.loc[
            (df_agg["dataset"] == dataset) & (df_agg["metric"] == metric)
        ].rename(columns={"reinit": "re-initialise"})
        plot(
            plot_data,
            800,
            0.5 if dataset == "imdb" else 0.75,
            0.85 if dataset == "imdb" else 0.9,
            dataset.upper(),
            metric.title().replace("_", "-"),
        )